In [1]:
from scigraDB import ScigraDB, Scigra
import requests, json,rdflib
from rdflib import Graph, plugin, BNode, URIRef, Literal
from SPARQLWrapper import JSON, SPARQLWrapper  # type: ignore

# using graph parse directly 

In [2]:
def query_to_rdf_graph(results_json):
    """
    Converts SPARQL JSON results to an RDFLib Graph.
    """
    g = Graph()
    for result in results_json["results"]["bindings"]:
        s_value = result["subject"]["value"]
        p_value = result["predicate"]["value"]
        o_value = result["object"]["value"]

        s = BNode(s_value) if result["subject"]["type"] == "bnode" else URIRef(s_value)
        p = URIRef(p_value)
        
        if result["object"]["type"] == "bnode":
            o = BNode(o_value)
        elif result["object"]["type"] == "uri":
            o = URIRef(o_value)
        elif result["object"]["type"] == "literal":
            if "datatype" in result["object"]:
                o = Literal(o_value, datatype=URIRef(result["object"]["datatype"]))
            elif "xml:lang" in result["object"]:
                o = Literal(o_value, lang=result["object"]["xml:lang"])
            else:
                o = Literal(o_value)
        else:
            o = Literal(o_value)

        g.add((s, p, o))
    return g

In [3]:
g1=Scigra()

In [4]:
g1.load("../../../Ontology-matters/reasoned/platforms_dome_core_reasoned_Hermit.ttl", f="ttl")

In [5]:
print(f"len(g1.g)={len(g1.g)}, len(g1.dict)={len(g1.dict)}")

len(g1.g)=1211, len(g1.dict)=220


# using query, as in the code 

In [6]:
q = """
        SELECT ?subject ?predicate ?object
        WHERE {
           GRAPH <http://dome40.io/dataset/data/platforms_dome_core_reasoned_Hermit> {
               ?subject ?predicate ?object .
           }
          }
        """

In [7]:
params = {"query": q}

In [8]:
response = requests.get("http://localhost:3030/dataset/sparql", params=params, timeout=10)

In [9]:
data=response.json()

In [10]:
print(data)

{'head': {'vars': ['subject', 'predicate', 'object']}, 'results': {'bindings': [{'subject': {'type': 'uri', 'value': 'https://emmc.eu/semantics/evmpo/evmpo.ttl#agent'}, 'predicate': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}, 'object': {'type': 'uri', 'value': 'http://www.w3.org/2002/07/owl#Class'}}, {'subject': {'type': 'uri', 'value': 'http://purl.org/dc/terms/creator'}, 'predicate': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}, 'object': {'type': 'uri', 'value': 'http://www.w3.org/2002/07/owl#AnnotationProperty'}}, {'subject': {'type': 'uri', 'value': 'https://emmc.eu/semantics/evmpo/evmpo.ttl#information_content_entity'}, 'predicate': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}, 'object': {'type': 'uri', 'value': 'http://www.w3.org/2002/07/owl#Class'}}, {'subject': {'type': 'uri', 'value': 'http://purl.org/dc/terms/license'}, 'predicate': {'type': 'uri', 'value': 'http://www.w3.org/1999/

In [11]:
sdb=ScigraDB()

In [12]:
sdb.query = q

In [13]:
sdb.run_query()

In [14]:
g2=sdb.get_rdflib_graph()

In [15]:
g2

<Graph identifier=N3593dfaf5e6d4e67a327fd86334f5fee (<class 'rdflib.graph.Graph'>)>

In [16]:
len(g2) # we can use this and compare to the length from direct reading! 

1211

In [17]:
for s,p,o in g2: 
    print(f"s:{s}")

s:b9
s:https://dome40.eu/semantics/dome4.0_core#EXTRACT
s:https://dome40.eu/semantics/dome4.0_core#has_platform_description
s:b15
s:https://dome40.eu/semantics/dome4.0_core#RDF
s:https://dome40.eu/semantics/dome4.0_core#has_cost_per_unit
s:https://dome40.eu/semantics/dome4.0_core#SIMULATE
s:https://dome40.eu/semantics/dome4.0_core#NOMAD
s:https://dome40.eu/semantics/dome4.0_core#TRANSFORM
s:https://dome40.eu/semantics/dome4.0_core#action
s:https://dome40.eu/semantics/dome4.0_core#LIST_OF_DATA_ACTIONS
s:https://dome40.eu/semantics/dome4.0_core#has_offering_periodicity
s:https://dome40.eu/semantics/dome4.0_core#CIF
s:https://dome40.eu/semantics/dome4.0_core#get_type_service
s:https://dome40.eu/semantics/dome4.0_core#CHEMEO
s:b24
s:b10
s:https://dome40.eu/semantics/dome4.0_core#THE_OPEN_QUANTUM_MATERIALS
s:https://emmc.eu/semantics/evmpo/evmpo.ttl#agent
s:http://www.w3.org/2011/http-methods#HEAD
s:https://dome40.eu/semantics/dome4.0_core#PUBCHEM
s:https://dome40.eu/semantics/dome4.0_core#

In [18]:
for s,p,o in g1: 
    print(f"s:{s}")

s:https://dome40.eu/semantics/dome4.0_core#EXTRACT
s:https://dome40.eu/semantics/scenario/platforms#MATERIALSPROJECT
s:https://dome40.eu/semantics/dome4.0_core#has_platform_description
s:https://dome40.eu/semantics/dome4.0_core#operation
s:https://dome40.eu/semantics/dome4.0_core#TRANSFORM
s:https://dome40.eu/semantics/scenario/platforms#THE_OPEN_QUANTUM_MATERIALS
s:https://dome40.eu/semantics/dome4.0_core#action
s:https://dome40.eu/semantics/dome4.0_core#has_offering_periodicity
s:https://dome40.eu/semantics/dome4.0_core#has_operation_type
s:https://dome40.eu/semantics/dome4.0_core#CIF
s:https://dome40.eu/semantics/reasoned/platforms_dome_core_reasoned#
s:https://dome40.eu/semantics/dome4.0_core#standard_identifier
s:https://emmc.eu/semantics/evmpo/evmpo.ttl#agent
s:http://www.w3.org/2011/http-methods#HEAD
s:https://dome40.eu/semantics/dome4.0_core#PATENT
s:https://dome40.eu/semantics/dome4.0_core#IDS_API_SPECIFICATION
s:https://dome40.eu/semantics/reasoned/platforms_dome_core_reasone

In [25]:
from scigraDB import compare_graphs, dump_nt_sorted

In [26]:
c=compare_graphs(g1, g2)

The graphs are different.
Testing iso1==iso2: False
Present in both: 780
Present in first: 431
Present in second: 431
size of first 1211, size of second 1211

Only in second:


In [27]:
type(g1)

scigraDB.Scigra

In [28]:
type(g2)

rdflib.graph.Graph

In [38]:
dump_nt_sorted(c["in first"])

<https://dome40.eu/semantics/dome4.0_core#AIIDA_FORMAT> <http://www.w3.org/2004/02/skos/core#prefLabel> "AiiDA export file"^^<http://www.w3.org/2001/XMLSchema#string> .
<https://dome40.eu/semantics/dome4.0_core#CARTOGRAPHY> <http://www.w3.org/2004/02/skos/core#prefLabel> "Cartography"^^<http://www.w3.org/2001/XMLSchema#string> .
<https://dome40.eu/semantics/dome4.0_core#CAS_NUMBER> <http://www.w3.org/2000/01/rdf-schema#comment> "A number of five-to-ten digits (XX[XXXXX]-XX-X) assigned to chemical substances by the American Chemical Society"^^<http://www.w3.org/2001/XMLSchema#string> .
<https://dome40.eu/semantics/dome4.0_core#CAS_NUMBER> <http://www.w3.org/2000/01/rdf-schema#comment> "Chemical Abstracts Service Registry Number (CASRN, or CAS number)"^^<http://www.w3.org/2001/XMLSchema#string> .
<https://dome40.eu/semantics/dome4.0_core#CAS_NUMBER> <http://www.w3.org/2000/01/rdf-schema#label> "CAS Number"^^<http://www.w3.org/2001/XMLSchema#string> .
<https://dome40.eu/semantics/dome4.0_

In [41]:
for i in c["in first"].triples((URIRef("https://dome40.eu/semantics/dome4.0_core#AIIDA_FORMAT"),None, None)) :
    print(i[1], i[2])

http://www.w3.org/2004/02/skos/core#prefLabel AiiDA export file


In [40]:
for i in c["in second"].triples((URIRef("https://dome40.eu/semantics/dome4.0_core#AIIDA_FORMAT"),None, None)) :
    print(i[1], i[2])

http://www.w3.org/2004/02/skos/core#prefLabel AiiDA export file


In [42]:
for i in c["in both"].triples((URIRef("https://dome40.eu/semantics/dome4.0_core#AIIDA_FORMAT"),None, None)) :
    print(i[1], i[2])

http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://emmc.eu/semantics/evmpo/evmpo.ttl#annotation
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#NamedIndividual
http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://dome40.eu/semantics/dome4.0_core#file_format
http://www.w3.org/2000/01/rdf-schema#seeAlso https://aiida.readthedocs.io/projects/aiida-core/en/v2.0.0b1/internals/storage/sqlite_zip.html
http://www.w3.org/2004/02/skos/core#inScheme https://dome40.eu/semantics/dome4.0_core#LIST_OF_FILE_FORMATS


In [44]:
for i in g1.g.triples((URIRef("https://dome40.eu/semantics/dome4.0_core#AIIDA_FORMAT"),None, None)) :
    print(i[1], i[2])

http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#NamedIndividual
http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://dome40.eu/semantics/dome4.0_core#file_format
http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://emmc.eu/semantics/evmpo/evmpo.ttl#annotation
http://www.w3.org/2000/01/rdf-schema#seeAlso https://aiida.readthedocs.io/projects/aiida-core/en/v2.0.0b1/internals/storage/sqlite_zip.html
http://www.w3.org/2004/02/skos/core#inScheme https://dome40.eu/semantics/dome4.0_core#LIST_OF_FILE_FORMATS
http://www.w3.org/2004/02/skos/core#prefLabel AiiDA export file


In [45]:
for i in g2.triples((URIRef("https://dome40.eu/semantics/dome4.0_core#AIIDA_FORMAT"),None, None)) :
    print(i[1], i[2])

http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#NamedIndividual
http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://dome40.eu/semantics/dome4.0_core#file_format
http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://emmc.eu/semantics/evmpo/evmpo.ttl#annotation
http://www.w3.org/2004/02/skos/core#inScheme https://dome40.eu/semantics/dome4.0_core#LIST_OF_FILE_FORMATS
http://www.w3.org/2004/02/skos/core#prefLabel AiiDA export file
http://www.w3.org/2000/01/rdf-schema#seeAlso https://aiida.readthedocs.io/projects/aiida-core/en/v2.0.0b1/internals/storage/sqlite_zip.html
